<a href="https://colab.research.google.com/github/Womics/lab_rotation/blob/main/first_workat%20labroat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.stats import norm

# Equivalent of vPow in C++
def vPow(base, exp):
    return np.power(base, exp)

# Equivalent of ZeroOne in C++
def ZeroOne(x):
    return 1 - x

# Equivalent of ColSums_cpp in C++
def ColSums_cpp(X):
    return np.sum(X, axis=0)

# Equivalent of Pruning_cpp in C++
def Pruning_cpp(West, z):
    n = West.shape[1]
    for j in range(n):
        if z[j] == 1:
            West[:, j][West[:, j] < 0] = 0
        elif z[j] == 0:
            West[:, j][West[:, j] > 0] = 0
    return West

# Equivalent of EstimateConnections_cpp in C++
def EstimateConnections_cpp(Lambda, Eta, nitr):
    n = Lambda.shape[1]
    I = np.eye(n)
    Theta = np.random.rand(n, n)
    np.fill_diagonal(Theta, 0)

    for itr in range(nitr):
        West = Lambda @ (I - Theta)
        for i in range(n):
            for j in range(n):
                Theta[i, j] = norm.cdf(West[i, j] + Eta[i, j])
        np.fill_diagonal(Theta, 0)
        LT = Lambda @ Theta
        Lambda = Lambda - np.diag(np.diag(Lambda)) + np.diag(np.diag(LT))
    return West

# Displaying the defined functions to ensure they are correctly defined
[vPow(np.array([2, 3]), np.array([2, 3])),
 ZeroOne(np.array([0.2, 0.8])),
 ColSums_cpp(np.array([[1, 2], [3, 4]])),
 Pruning_cpp(np.array([[1, -1], [-1, 1]]), np.array([1, 0])),
 EstimateConnections_cpp(np.array([[0.1, 0.3], [0.2, 0.4]]), np.array([[0.05, 0.07], [0.02, 0.06]]), 1)]


[array([ 4, 27]),
 array([0.8, 0.2]),
 array([4, 6]),
 array([[ 1, -1],
        [ 0,  0]]),
 array([[0.08189437, 0.24319524],
        [0.17585916, 0.28639048]])]

In [5]:
!pip install scipy

In [7]:
# Re-defining the previously attempted functions with corrections

def CalculateSpikeProbMatrix(pre_spike, post_spike, nclust):
    # Calculate probability of spiking and resting based on input spike counts
    prob_fire = np.mean(pre_spike, axis=0)
    prob_rest = np.mean(post_spike, axis=0)
    return prob_fire, prob_rest

# Probit transformation using scipy's norm.ppf (inverse of cdf for normal distribution)
def ProbitTransformation(matrix):
    return norm.ppf(matrix)

# Gibbs sampling for statistical estimation (placeholder for specific sampling logic)
def GibbsSampling(Lambda, Eta, nitr_em, nsamp, excl_ratio):
    samples = np.random.normal(loc=Lambda, scale=np.sqrt(Eta), size=(nsamp, Lambda.shape[0], Lambda.shape[1]))
    valid_samples = samples[int(nsamp * excl_ratio):]  # Apply exclusion ratio
    return np.mean(valid_samples, axis=0)

# Estimate neural connections with probabilistic approach
def EstimateConnections(Lambda, Eta, z, param):
    n = Lambda.shape[1]
    Theta = np.random.rand(n, n)
    np.fill_diagonal(Theta, 0)
    for _ in range(param['number_of_iteration']):
        for i in range(n):
            for j in range(n):
                Theta[i, j] = z[i] * z[j] * norm.cdf(Lambda[i, j] - Eta[i, j])
        np.fill_diagonal(Theta, 0)
    return Theta


In [12]:
# Fixing the issue in EstimateExcitatoryLabelGibbs by correctly handling dimensions

def EstimateExcitatoryLabelGibbs(Lambda, Eta, proj_type, param):
    # Correct dimensioning for Alpha array
    Alpha = np.zeros((param['number_of_alpha'], Lambda.shape[0], Lambda.shape[1]))
    for i in range(param['number_of_alpha']):
        Alpha[i] = GibbsSampling(Lambda, Eta, param['number_of_EM_iteration'], param['number_of_Gibbs_sampler'], param['gibbs_exclude_ratio'])
    # Determine labels by averaging across samples and applying a threshold
    return np.mean(Alpha, axis=0) > 0.5
# Re-test functions with example data
Lambda_example = np.array([[0.1, 0.2], [0.3, 0.4]])#Lambda_test = np.array([[0.6, 0.7], [0.8, 0.9]])ここをいじると接続が結構変わる
Eta_example = np.array([[0.05, 0.07], [0.06, 0.08]])#Eta_test = np.array([[0.1, 0.2], [0.3, 0.4]])ここをいじっても接続が変わるみたい
param_example = {'number_of_iteration': 10, 'number_of_alpha': 2, 'number_of_Gibbs_sampler': 100, 'gibbs_exclude_ratio': 0.02, 'number_of_EM_iteration': 50}

# Calculate and display example outputs
spike_prob_fire, spike_prob_rest = CalculateSpikeProbMatrix(np.array([[1, 2], [3, 4]]), np.array([[2, 1], [4, 3]]), 2)
probit_example = ProbitTransformation(np.array([0.25, 0.75]))
gibbs_samples = GibbsSampling(Lambda_example, Eta_example, 50, 100, 0.02)
theta_estimates = EstimateConnections(Lambda_example, Eta_example, np.array([1, 0]), param_example)
excitatory_labels_corrected = EstimateExcitatoryLabelGibbs(Lambda_example, Eta_example, 'e', param_example)

excitatory_labels_corrected

array([[False, False],
       [False, False]])